In [50]:
import pandas as pd
import numpy as np
import os
from pathlib import Path


df_inventory = pd.read_csv('Inventory_mikelele_2025.May.30.csv')
# display(df_inventory.loc[(df_inventory['Name'] == 'Lightning Bolt')])

df_inventory = df_inventory.drop(
    columns=[
        'Tradelist Count',
        'Decks Count Built',
        'Decks Count All',
        'Type',
        'Cost',
        'Rarity',
        'Price',
        'Image URL',
        'Edition',
        'Edition Code',
        'Card Number',
        'Condition',
        'Language',
        'Foil',
        'Signed',
        'Artist Proof',
        'Altered Art',
        'Misprint',
        'Promo',
        'Textless',
        'Printing Id', 
        'Printing Note',
        'Tags',
        'My Price',
        'Last Updated',
        'TcgPlayer ID'
        ])
display(df_inventory.head())

display(df_inventory.loc[df_inventory['Name'] == 'Lightning Bolt'])

df_inventory_group = df_inventory.groupby('Name', as_index=False).agg({'Count': 'sum', 'Scryfall ID': 'first'}).rename(columns={'Count': 'sum_count'})[['Name', 'sum_count', 'Scryfall ID']]
display(df_inventory_group.loc[df_inventory_group['Name'] == 'Lightning Bolt'])

,Count,Name,Scryfall ID
0,4,Abandoned Campground,ee0565f5-ebdb-43f9-bbb4-0485b1968937
1,3,Academy Drake,f8bacb12-da46-4b00-804f-9ff6bff452bc
2,4,Academy Journeymage,a46a65e0-66a3-4896-8acc-0ad5e9927c40
3,2,Access Tunnel,edf8eb51-9643-4c54-b38e-e7abea92bbe1
4,4,Accumulated Knowledge,ad88e5ee-0eee-47af-a7b4-9bac044e1c8c


,Count,Name,Scryfall ID
1355,4,Lightning Bolt,ccee0b4c-0cb0-4c0f-8ddc-bc74b8b97273
1356,4,Lightning Bolt,cb9b9a9d-ae4c-4e04-bf9d-cae48f01292c
1357,1,Lightning Bolt,9521375e-0bc1-45ef-b513-6d332a25f9d2


,Name,sum_count,Scryfall ID
1295,Lightning Bolt,9,ccee0b4c-0cb0-4c0f-8ddc-bc74b8b97273


In [51]:
directory = Path('decks')

df_decks = pd.DataFrame()

for file in directory.iterdir():

    if file.is_file():
        deck_name = file.name.split('_')[0]
        temp_df = pd.read_csv(file)
        deck_name_arr = list([deck_name] * len(temp_df))
        temp_df.insert(0, 'deck', deck_name_arr)

        temp_df = temp_df.drop(columns=['Rarity', 'Section', 'Last Updated', 'TcgPlayer ID'])
        #display(temp_df.head())
        df_decks = pd.concat([df_decks, temp_df])


# display(df_decks)

# print(df_decks['deck'].unique())

display(df_decks.loc[df_decks['Name'] == 'Lightning Bolt'])

cards_in_decks = df_decks.groupby('Name')['Count'].sum().sort_values(ascending=False)
print(cards_in_decks)

,deck,Count,Name,Type,Cost,Price,Image URL,Scryfall ID
9,Goblins,4,Lightning Bolt,Instant,{R},$0.85,https://s.deckbox.org/system/images/mtg/cards/...,77c6fa74-5543-42ac-9ead-0e890b188e99
7,Kiln Fiend,2,Lightning Bolt,Instant,{R},$0.85,https://s.deckbox.org/system/images/mtg/cards/...,77c6fa74-5543-42ac-9ead-0e890b188e99
7,Rakdos Madness,4,Lightning Bolt,Instant,{R},$0.85,https://s.deckbox.org/system/images/mtg/cards/...,77c6fa74-5543-42ac-9ead-0e890b188e99
8,Rakdos Weenie Sac Burn,4,Lightning Bolt,Instant,{R},$0.85,https://s.deckbox.org/system/images/mtg/cards/...,77c6fa74-5543-42ac-9ead-0e890b188e99
6,Red Burn,4,Lightning Bolt,Instant,{R},$0.85,https://s.deckbox.org/system/images/mtg/cards/...,77c6fa74-5543-42ac-9ead-0e890b188e99


Name
Mountain                  65
Island                    57
Forest                    48
Swamp                     47
Plains                    34
                          ..
Environmental Sciences     1
Blood Celebrant            1
Blazing Volley             1
Azorius Guildgate          1
Ardent Elementalist        1
Name: Count, Length: 374, dtype: int64


In [53]:
def cards_needed(card_name):

    if card_name not in df_decks['Name'].values:
        print("That card is not in any of your decks. Perhaps check spelling")
        return
    
    total_needed = df_decks[df_decks['Name'] == card_name]['Count'].sum()

    if card_name not in df_inventory_group['Name'].values:
        total_inventory = 0
    else:
        total_inventory = df_inventory_group[df_inventory_group['Name'] == card_name]['sum_count'].sum()
        
    card_price_string = df_decks[df_decks['Name'] == card_name]['Price'].unique()[0]
    card_price = float(card_price_string.split('$')[1])

    print(f"Summary for {card_name}")
    print(f"You have a total of {total_inventory}")
    print(f"You need {total_needed}")
    print(f"You need to buy {total_needed - total_inventory}")
    print(f"Lowest Price is {card_price}")
    print(f"This will cost you ~ {card_price * (total_needed - total_inventory)}€")


# card_input = input("Enter a card name")
# cards_needed(card_input)

In [54]:
df_merged = pd.merge(df_decks, df_inventory_group, how='left', on='Name').drop(columns='Scryfall ID_y').fillna(0)

df_merged['sum_count'] = df_merged['sum_count'].astype(int)
def to_complete_deck(deck_name):

    df_deck = df_merged[df_merged['deck'] == deck_name]
    df_deck = df_deck.drop(columns=['Image URL', 'Scryfall ID_x'])
    df_deck['needed'] = df_deck['Count'] - df_deck['sum_count']
    df_deck_needed = df_deck.loc[df_deck['needed'] > 0]
    

    df_deck_needed['price_clean'] = df_deck_needed['Price'].str.replace('$', '').astype(float)
    df_deck_needed['cost'] = df_deck_needed['price_clean'] * df_deck_needed['needed']

    total_cost = round(df_deck_needed['cost'].sum(), 2)
    display(df_deck_needed)
    print(f"You need {total_cost}€ to complete the deck")


# deck_input = input("Enter a deck name")
# to_complete_deck(deck_input)

In [ ]:
df_deck_cards = df_decks.groupby('Name').agg({'Count': 'sum', 'Price': 'min'}).reset_index().sort_values('Count', ascending=False)

lands_to_drop = ['Forest', 'Mountain', 'Island', 'Plains', 'Swamp']

df_deck_cards = df_deck_cards[~df_deck_cards['Name'].isin(lands_to_drop)]

total_needed = pd.merge(df_deck_cards, df_inventory_group, how='left', on='Name').drop(columns='Scryfall ID').fillna(0)

total_needed.rename(columns={'sum_count': 'Total'}, inplace=True)
total_needed['Total'] = total_needed['Total'].astype(int)
total_needed['Price'] = total_needed['Price'].str.replace('$', '').astype(float)
total_needed['Needed'] = total_needed['Count'] - total_needed['Total']
total_needed['Needed'] = np.where(total_needed['Needed'] < 0, 0, total_needed['Needed'])

display(total_needed.sort_values('Price', ascending=False))

print(f"You need a total of {total_needed['Needed'].sum()} cards to complete your Battle Box")
print(f"Estimated minimum cost: {round((total_needed['Needed'] * total_needed['Price']).sum(), 2)}€")
print(f"The most expensive card is {total_needed['Price'].max()}")




,Name,Count,Price,Total,Needed
31,Lotus Petal,8,22.05,0,8
147,Cabal Ritual,4,12.57,0,4
49,Dust to Dust,7,5.01,0,7
201,Mental Note,4,5.01,0,4
38,Manamorphose,8,4.93,4,4
...,...,...,...,...,...
312,Feat of Resistance,2,0.02,2,0
23,Balemurk Leech,8,0.02,7,1
131,Grand Entryway // Elegant Rotunda,4,0.02,4,0
234,Walking Sarcophagus,4,0.01,0,4


You need a total of 763 cards to complete your Battle Box
Estimated minimum cost: 803.59€
The most expensive card is 22.05


,deck,Count,Name,Type,Cost,Price,Image URL,Scryfall ID
